<a href="https://colab.research.google.com/github/Reptilefury/coursera-machine-learning/blob/main/SavingTheModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf


In [3]:
from tensorflow import keras

In [4]:
#We start by loading the dataset 
mnist = tf.keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [5]:
#Split the dataset into training and testing
(x_train,y_train),(x_test,y_test) = mnist

In [6]:
#Normalize the data by turning it into floating point numbers in between 0 and 1
x_train , y_train = x_train/255.0 , y_train/255.0
x_test,y_test = x_test/255.0 , y_test/255.0

In [7]:
#We create a sequential model 
model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=x_train[0].shape),
      tf.keras.layers.Dense(300,activation="relu"),
      tf.keras.layers.Dense(100,activation="relu"),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10)#10 neurons in the output layer because we have 10  classes to activate
])

In [8]:
predictions = model.predict(x_train[:1])

In [9]:
#Softmax function 
tf.nn.softmax(predictions)


<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[0.12625219, 0.0678308 , 0.0933851 , 0.03655002, 0.11998049,
        0.03614298, 0.03660029, 0.19252545, 0.14673345, 0.14399917]],
      dtype=float32)>

In [10]:
#We create a loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [11]:
loss_fn(y_test[3], predictions).numpy()

2.0694737

In [12]:
#We declare hyperparameters
model.compile(loss = loss_fn, optimizer='sgd',metrics=['accuracy'])

In [13]:
history = model.fit(x_train, y_train, epochs=5, callbacks=[])

Epoch 1/5
1875/1875 [==============================] - 9s 4ms/step - loss: 0.0117 - accuracy: 0.0986
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 4.2846e-04 - accuracy: 0.0987
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 2.4147e-04 - accuracy: 0.0987
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 1.6686e-04 - accuracy: 0.0987
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 1.2565e-04 - accuracy: 0.0987


In [14]:
#Saving the model is as simple as 
model.save("MyModel.h5")

In [15]:
#Loading a saved model
model = tf.keras.models.load_model("MyModel.h5")

In [16]:
#We should implement early Stopping or checkpoints to save our model best parameters just incase our model starts to overfit or when there is no progress in the validation loss
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("MyModel.h5", save_best_only=True)

In [17]:
#The model.fit method has callback functions that we can pass a list of objects to call during , before and after training
#In this case we call the callback and pass in the checkpoint method to prevent our model from overfitting
#Another default callback called after every epoch is the validation and training loss to gauge our model perfomance after every training cycle or epoch
 
model.fit(x_train, y_train, epochs = 5,  callbacks=[checkpoint_cb])

Epoch 1/5
1875/1875 [==============================] - 12s 6ms/step - loss: 9.9998e-05 - accuracy: 0.0987
Epoch 2/5
1875/1875 [==============================] - 10s 5ms/step - loss: 8.6002e-05 - accuracy: 0.0987
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 7.2079e-05 - accuracy: 0.0987
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 6.2040e-05 - accuracy: 0.0987
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 5.7350e-05 - accuracy: 0.0987


In [ ]:
#Early stopping method will stop training when it measures no progress on the validation set
#Then it will roll back to the best model parameters
#The number of epochs can be set to a large value since the model will automatically stop training
EarlyStopping = tf.keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)
model.fit(x_train,y_train, epochs=100, callbacks=[checkpoint_cb,EarlyStopping])

#EarlyStopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [19]:
#We could create custom callbacks, to display the ratio between the validation loss and training loss to detect overfitting 

class PrintValCallBack(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,log):
    print("\nval:/train{:2f".format(logs["val_logs"]/logs["loss"]))

    #We could also call onTrain begin onTrain end onbatch begin on bacth end 
    #We could also use callbacks when Debugging

    #Using callbacks

Visualising using Tensorboard

In [20]:
import os 
root_logdir=os.path.join(os.curdir,"MyModel") 
def get_run_logdir():
  import time
  run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S") 
  return os.path.join(root_logdir,run_id)

In [21]:
run_logdir = get_run_logdir()

In [22]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)

In [23]:
history = model.fit(x_train, y_train, callbacks=[tensorboard_cb])

1875/1875 [==============================] - 8s 4ms/step - loss: 4.3057e-05 - accuracy: 0.0987


In [24]:
#Visualising using tensorboard
import os
root_rundir = os.path.join(os.curdir, "MyModel")
def get_run_logdir():
  import time
  run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
  return os.path.join(root_dir, run_id)

In [25]:
run_dir = get_run_logdir()

In [26]:
#Create a tensorboard callback
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_dir)


In [27]:
history = model.fit(x_train,y_train,epochs=5,callbacks=[tensorboard_cb])

Epoch 1/5
1875/1875 [==============================] - 10s 6ms/step - loss: 3.9566e-05 - accuracy: 0.0987
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 4.0195e-05 - accuracy: 0.0987
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 3.4617e-05 - accuracy: 0.0987
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 3.3122e-05 - accuracy: 0.0987
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 2.9891e-05 - accuracy: 0.0987


In [28]:
#Fine tuning Hyperparameters

In [32]:
#Visualisation using tensorBoard
import os
root_dir =os.path.join(os.curdir, "MyLogs")
#Define a method to return the location of the logs
def get_dir():
  import time
  run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
  return os.path.join(root_dir, run_id)

In [33]:
#We save the method into a variable to access it later
root = get_dir()

In [34]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(root)

In [36]:
#When we call the model.fit method we can now pass in the tensorboard_cb 
history = model.fit(x_train, y_train, epochs=5, callbacks=[checkpoint_cb, tensorboard_cb])

Epoch 1/5
1875/1875 [==============================] - 10s 6ms/step - loss: 2.9776e-05 - accuracy: 0.0987
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 2.6807e-05 - accuracy: 0.0987
Epoch 3/5
1875/1875 [==============================] - 11s 6ms/step - loss: 2.5384e-05 - accuracy: 0.0987
Epoch 4/5
1875/1875 [==============================] - 12s 6ms/step - loss: 2.3869e-05 - accuracy: 0.0987
Epoch 5/5
1875/1875 [==============================] - 12s 6ms/step - loss: 2.2064e-05 - accuracy: 0.0987


In [39]:
history = model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, to_file="MyModel.png")

In [52]:
#Cross validation 
def build_model(n_hidden=1, n_neurons=30,learning_rate=3e-3, input_shape=[8]):
  model = tf.keras.Sequential()
  options = {"input_shape":input_shape}
  for layer in n_hidden:
    model.add(tf.keras.layers.Dense(n_neurons,activation="relu",**options))
    options = {}
    model.add(tf.keras.layers.Dense(1,**options))
    optimizer = tf.keras.optimizers.SGD(learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [55]:
#Univariate regression i.e one output neuron/output
#Cross validation: We start by creating a sequential mode
def build_model(n_hidden=1,n_neurons=30,learning_rate=3e-3,input_shape=[8]):
  #create an instance of the class Sequential
  model = tf.keras.Sequential()
  #We define the the input shape in a dictionary
  options = {"input_shape":input_shape}
  for layer in n_hidden:
    model.add(n_neurons, activations="relu", **options)
    options ={}
    model.add(tf.keras.layers.Dense(1,**options))
    optimizer = tf.keras.optimizers.SGD(learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

In [56]:
keras_reg = tf.keras.wrappers.scikit_learn.KerasRegressor(build_model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [ ]:
#We use the fit() method to train, we use the score() method to evaluate it and the predict() method to predict 
keras_reg.fit(x_train, y_train, epochs=20,  callbacks=[tf.keras.callbacks.EarlyStopping(patience=10,)]) #Would pass in validation data
m_test =  keras_reg.score(x_valid, y_valid)
predictions = keras_reg.predict(x_new)

In [ ]:
#REDO